In [1]:
import json
import os.path

from uuid import uuid4
from datetime import datetime
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport
from gql.transport.requests import RequestsHTTPTransport

import graphql
from graphql import get_introspection_query, build_client_schema
import requests

from csid import CsidUtility
from simple_graphql import CoreDataGraphQlClient

import pdb

###### Basic Setup

In [2]:
environment = 'PRODUCTION'  # or 'PRODUCTION'
application_name = 'CoreDataUAT' if environment == 'UAT' else 'CoreData'

###### Get CSID

In [3]:
csid_utility = CsidUtility(environment=environment, application_name=application_name)
csid_token = csid_utility.get_csid_token()
print(f"CSID Token: {csid_token}")


Reading CSID token from file ./production-csid_token.txt...
CSID token has expired, retrieving a new one from the API.
Retrieving PRODUCTION CSID token for CoreData from API...
CSID Token: eyJhbGciOiJSUzI1NiIsImtpZCI6IjY0QTBFQkI4M0UwMTUwNDFFQkUyQjc3QUQxMDg0REZFOTEwRDc2OUYiLCJ0eXAiOiJKV1QifQ.eyJqdGkiOiI3MzdiYTAzZi1jNmExLTQyYWYtYTFhOC0wOTc4NzgzZjQ4NzUiLCJzdWIiOiJBRFxcem9uZyIsImdpdmVuX25hbWUiOiJaaGkgWGlhbiIsImZhbWlseV9uYW1lIjoiT25nIiwiZW1haWwiOiJaaGlYaWFuLk9uZ0BtbHAuY29tIiwiaHR0cDovL3NjaGVtYXMueG1sc29hcC5vcmcvd3MvMjAwNS8wNS9pZGVudGl0eS9jbGFpbXMvbmFtZSI6IkFEXFx6b25nIiwiSXNTZXJ2aWNlQWNjb3VudCI6ZmFsc2UsIkNvcmVEYXRhLkFwcGlhbkVudGl0bGVtZW50T3ZlcnJpZGVzIjpbIlZpZXdBbGxQb3J0Zm9saW9zIiwiVmlld1BuTENvbHVtbnMiXSwiQ29yZURhdGEuQ29sdW1uQ2hvb3NlciI6IlNlbGVjdEFsbCIsIkNvcmVEYXRhLkRhdGFzZXRzIjpbIkFjY291bnQiLCJBY2NvdW50RGV0YWlsIiwiQWNjb3VudERldGFpbEhpc3RvcnkiLCJBY2NvdW50SGlzdG9yeSIsIkFjY291bnRQb3J0Zm9saW8iLCJBY2NydWFsUG5MRmluYWxUYWciLCJBY2NydWFsUG5MVGFnIiwiQXNzZXRTZXJ2aWNpbmciLCJDYXNoZmxvdyIsIkNhc2hHcm91cFBu

###### SETUP NETWORK PROXY

In [4]:
proxy_url = "http://siproxyvip.ad.mlp.com:3128"
headers = {
    "Authorization": f"Bearer {csid_token}",
}

###### MAIN

In [5]:
gql_client = CoreDataGraphQlClient(environment=environment)
schema = gql_client.get_schema(csid_token=csid_token)
print('schema', schema)

Retrieving GraphQL schema...
Executing introspection query to retrieve schema...
Building client schema from introspection result...
Schema retrieval complete.
schema <graphql.type.schema.GraphQLSchema object at 0x0000000004272D40>


In [6]:
str(dir(schema))

"['__annotations__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_implementations_map', '_sub_type_map', '_validation_errors', 'ast_node', 'description', 'directives', 'extension_ast_nodes', 'extensions', 'get_directive', 'get_implementations', 'get_possible_types', 'get_root_type', 'get_type', 'is_sub_type', 'mutation_type', 'query_type', 'subscription_type', 'to_kwargs', 'type_map', 'validation_errors']"

In [7]:
#schema.get_root_type(graphql.language.OperationType.QUERY)
#schema.get_root_type(graphql.language.OperationType.MUTATION)
#schema.get_root_type(graphql.language.OperationType.SUBSCRIPTION)
# Note: CoreData GraphQL API only has QUERY and SUBSCRIPTION (it does not have MUTATION)

str(dir(schema.query_type))
str(dir(schema.subscription_type))

"['__annotations__', '__class__', '__copy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_fields', '_interfaces', 'ast_node', 'description', 'extension_ast_nodes', 'extensions', 'fields', 'interfaces', 'is_type_of', 'name', 'to_kwargs']"

In [8]:
str(dir(schema.subscription_type.fields))

"['__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__ior__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__ror__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'copy', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'setdefault', 'update', 'values']"

In [9]:
sorted(schema.subscription_type.fields.keys())

['AccountDetailHistories',
 'AccountDetails',
 'AccountHistories',
 'AccountPortfolios',
 'Accounts',
 'AssetServicings',
 'CashGroupPnLTags',
 'CashPnLTags',
 'CashPositionAccounts',
 'CashPositionFinalAccounts',
 'CashPositionFinalLinkIds',
 'CashPositionLinkIds',
 'CashPositionPrePostSweepSummaries',
 'CashPositionPrePostSweeps',
 'Cashflows',
 'Confirmations',
 'CorporateActionElections',
 'CorporateActions',
 'DividendAccruals',
 'EDSDates',
 'EndurPositions',
 'ExecutingBrokerCodes',
 'FeeCommissionAttributions',
 'GenevaCashflows',
 'GenevaTrades',
 'GenevaTransactions',
 'ImaginePositions',
 'InternalTransferTransactionIds',
 'MurexPositions',
 'PortfolioHistories',
 'Portfolios',
 'PositionAnalytics',
 'PositionCashSweepMaps',
 'PositionPnLTags',
 'PositionSensitivities',
 'Positions',
 'ProductMapUpdates',
 'Products',
 'RepoExposures',
 'Securities',
 'SecurityDetails',
 'SecurityHistories',
 'SettleDatePositions',
 'SettledCashPositionAccounts',
 'SettledCashPositionFinalAc

In [20]:
graphql_query = """
query {
  Trades(
    page: {limit: 10}
    where: {
      tradeDate: {EQ: "2025-06-16"}
    }
  ) {
        select {
            tradeDate
            source
            upstreamPositionId
            positionCurrency
            security { ric }
            account { clearingBrokerCode mainAccountCode }
        }
    }
}
"""

In [21]:
#query_result = gql_client.query(graphql_query, csid_token=csid_token)
query_result = await gql_client.query(graphql_query, csid_token=csid_token)
print('query_result', query_result)

query_result {'Trades': {'select': [{'tradeDate': '2025-06-16', 'source': 'Endur', 'upstreamPositionId': '5644944', 'positionCurrency': 'USD', 'security': {'ric': 'SM65FQ5'}, 'account': {'clearingBrokerCode': 'SGAS', 'mainAccountCode': '075-86851'}}, {'tradeDate': '2025-06-16', 'source': 'Endur', 'upstreamPositionId': '5644944', 'positionCurrency': 'USD', 'security': {'ric': 'SM65FQ5'}, 'account': {'clearingBrokerCode': 'SGAS', 'mainAccountCode': '075-86851'}}, {'tradeDate': '2025-06-16', 'source': 'Endur', 'upstreamPositionId': '5644945', 'positionCurrency': 'USD', 'security': {'ric': 'SZZFQ5'}, 'account': {'clearingBrokerCode': 'SGAS', 'mainAccountCode': '075-86851'}}, {'tradeDate': '2025-06-16', 'source': 'Endur', 'upstreamPositionId': '5644946', 'positionCurrency': 'USD', 'security': {'ric': 'SZZFN5'}, 'account': {'clearingBrokerCode': 'JPSL', 'mainAccountCode': '12575'}}, {'tradeDate': '2025-06-16', 'source': 'Endur', 'upstreamPositionId': '5644946', 'positionCurrency': 'USD', 'se

In [22]:
str(dir(schema.subscription_type.fields['Trades']))

"['__annotations__', '__class__', '__copy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'args', 'ast_node', 'deprecation_reason', 'description', 'extensions', 'resolve', 'subscribe', 'to_kwargs', 'type']"

In [23]:
fields = schema.subscription_type.fields['Positions'].type.fields['select'].type.of_type.fields
len(fields)

94

In [24]:
fields['account'].type.fields['accountSubtype'].type.name
fields['account'].type.name

'Account'

In [25]:
dir(fields['account'].type.fields['effectiveStartTimestamp'])
#fields['account'].type.fields
dir(fields['account'].type.fields['effectiveStartTimestamp'].type)

['__annotations__',
 '__class__',
 '__copy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'ast_node',
 'description',
 'extension_ast_nodes',
 'extensions',
 'name',
 'parse_literal',
 'parse_value',
 'serialize',
 'specified_by_url',
 'to_kwargs']

In [26]:
sorted(fields['account'].type.fields)

['accountStatus',
 'accountSubtype',
 'accountType',
 'accountVersion',
 'agreementClass',
 'agreementCode',
 'agreementType',
 'agreementTypeDesc',
 'baseAgreement',
 'baseAgreementType',
 'cashSwap',
 'clearingBrokerCode',
 'clearingBrokerName',
 'effectiveEndTimestamp',
 'effectiveStartTimestamp',
 'fundFamilyCode',
 'mainAccountCode',
 'mainAccountId',
 'mlpLegalEntity',
 'reconAccountGroup',
 'streetAccountBrokerCode',
 'streetAccountBrokerName',
 'streetAccountCode',
 'streetAccountLegalEntity',
 'streetAccountLegalEntityFundFamilyCode',
 'streetAccountType',
 'streetLegalEntityCode',
 'ultimateParentCode',
 'ultimateParentName']

In [27]:
print(fields['account'].type.to_kwargs())

{'name': 'Account', 'description': 'Represents the most current view of Appian Main Account ref data', 'extensions': {}, 'ast_node': None, 'extension_ast_nodes': (), 'fields': {'accountSubtype': <GraphQLField <GraphQLScalarType 'String'>>, 'streetAccountCode': <GraphQLField <GraphQLScalarType 'String'>>, 'effectiveEndTimestamp': <GraphQLField <GraphQLScalarType 'SqlTimestamp'>>, 'accountStatus': <GraphQLField <GraphQLScalarType 'String'>>, 'agreementCode': <GraphQLField <GraphQLScalarType 'String'>>, 'ultimateParentCode': <GraphQLField <GraphQLScalarType 'String'>>, 'streetAccountType': <GraphQLField <GraphQLScalarType 'String'>>, 'accountVersion': <GraphQLField <GraphQLScalarType 'String'>>, 'cashSwap': <GraphQLField <GraphQLScalarType 'String'>>, 'accountType': <GraphQLField <GraphQLScalarType 'String'>>, 'baseAgreementType': <GraphQLField <GraphQLScalarType 'String'>>, 'mainAccountCode': <GraphQLField <GraphQLScalarType 'String'>>, 'clearingBrokerName': <GraphQLField <GraphQLScalarT

In [28]:
# fields = schema.subscription_type.fields['Positions'].type.fields['select'].type.of_type.fields
trades_fields = schema.subscription_type.fields['Trades'].type.fields['select'].type.of_type.fields

In [29]:
trades_fields

{'repoInterest': <GraphQLField <GraphQLScalarType 'BigDecimal'>>,
 'lastTranStatus': <GraphQLField <GraphQLScalarType 'String'>>,
 'debtorExternalAccount': <GraphQLField <GraphQLScalarType 'String'>>,
 'eventRelatedTradeType': <GraphQLField <GraphQLScalarType 'String'>>,
 'fixingRate': <GraphQLField <GraphQLScalarType 'BigDecimal'>>,
 'effectiveEndTimestamp': <GraphQLField <GraphQLScalarType 'SqlTimestamp'>>,
 'securityId': <GraphQLField <GraphQLScalarType 'String'>>,
 'repoHaircutAmount': <GraphQLField <GraphQLScalarType 'BigDecimal'>>,
 'previousTradeId': <GraphQLField <GraphQLScalarType 'BigInteger'>>,
 'positionCurrency': <GraphQLField <GraphQLScalarType 'String'>>,
 'securitySource': <GraphQLField <GraphQLScalarType 'String'>>,
 'sellerName': <GraphQLField <GraphQLScalarType 'String'>>,
 'securityDescription': <GraphQLField <GraphQLScalarType 'String'>>,
 'netProceeds': <GraphQLField <GraphQLScalarType 'BigDecimal'>>,
 'settlementDate': <GraphQLField <GraphQLScalarType 'Date'>>,
 

In [30]:
schema.subscription_type.fields

{'SettledCashPositionFinalLinkIds': <GraphQLField <GraphQLObjectType 'SettledCashPositionFinalLinkIdSubscription'>>,
 'Cashflows': <GraphQLField <GraphQLObjectType 'CashflowSubscription'>>,
 'AccountPortfolios': <GraphQLField <GraphQLObjectType 'AccountPortfolioSubscription'>>,
 'PortfolioHistories': <GraphQLField <GraphQLObjectType 'PortfolioHistorySubscription'>>,
 'SecurityHistories': <GraphQLField <GraphQLObjectType 'SecurityHistorySubscription'>>,
 'ExecutingBrokerCodes': <GraphQLField <GraphQLObjectType 'ExecutingBrokerCodeSubscription'>>,
 'PositionSensitivities': <GraphQLField <GraphQLObjectType 'PositionSensitivitySubscription'>>,
 'CashPositionPrePostSweeps': <GraphQLField <GraphQLObjectType 'CashPositionPrePostSweepSubscription'>>,
 'Portfolios': <GraphQLField <GraphQLObjectType 'PortfolioSubscription'>>,
 'PositionPnLTags': <GraphQLField <GraphQLObjectType 'PositionPnLTagSubscription'>>,
 'InternalTransferTransactionIds': <GraphQLField <GraphQLObjectType 'InternalTransferTr

In [40]:
sorted(schema.subscription_type.fields.items(), key=lambda x: type(x[1]).__name__)

[('SettledCashPositionFinalLinkIds',
  <GraphQLField <GraphQLObjectType 'SettledCashPositionFinalLinkIdSubscription'>>),
 ('Cashflows', <GraphQLField <GraphQLObjectType 'CashflowSubscription'>>),
 ('AccountPortfolios',
  <GraphQLField <GraphQLObjectType 'AccountPortfolioSubscription'>>),
 ('PortfolioHistories',
  <GraphQLField <GraphQLObjectType 'PortfolioHistorySubscription'>>),
 ('SecurityHistories',
  <GraphQLField <GraphQLObjectType 'SecurityHistorySubscription'>>),
 ('ExecutingBrokerCodes',
  <GraphQLField <GraphQLObjectType 'ExecutingBrokerCodeSubscription'>>),
 ('PositionSensitivities',
  <GraphQLField <GraphQLObjectType 'PositionSensitivitySubscription'>>),
 ('CashPositionPrePostSweeps',
  <GraphQLField <GraphQLObjectType 'CashPositionPrePostSweepSubscription'>>),
 ('Portfolios', <GraphQLField <GraphQLObjectType 'PortfolioSubscription'>>),
 ('PositionPnLTags',
  <GraphQLField <GraphQLObjectType 'PositionPnLTagSubscription'>>),
 ('InternalTransferTransactionIds',
  <GraphQLField

In [ ]:
]

dict_items([('SettledCashPositionFinalLinkIds', <GraphQLField <GraphQLObjectType 'SettledCashPositionFinalLinkIdSubscription'>>), ('Cashflows', <GraphQLField <GraphQLObjectType 'CashflowSubscription'>>), ('AccountPortfolios', <GraphQLField <GraphQLObjectType 'AccountPortfolioSubscription'>>), ('PortfolioHistories', <GraphQLField <GraphQLObjectType 'PortfolioHistorySubscription'>>), ('SecurityHistories', <GraphQLField <GraphQLObjectType 'SecurityHistorySubscription'>>), ('ExecutingBrokerCodes', <GraphQLField <GraphQLObjectType 'ExecutingBrokerCodeSubscription'>>), ('PositionSensitivities', <GraphQLField <GraphQLObjectType 'PositionSensitivitySubscription'>>), ('CashPositionPrePostSweeps', <GraphQLField <GraphQLObjectType 'CashPositionPrePostSweepSubscription'>>), ('Portfolios', <GraphQLField <GraphQLObjectType 'PortfolioSubscription'>>), ('PositionPnLTags', <GraphQLField <GraphQLObjectType 'PositionPnLTagSubscription'>>), ('InternalTransferTransactionIds', <GraphQLField <GraphQLObjectTy